<a href="https://colab.research.google.com/github/myra349/vistarnetra/blob/main/Final_year_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final year project space management





Here, the seats are arranged at a fixed distance.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrow
from shapely.geometry import Polygon, box

# -----------------------------------------------------------
# ASK USER SEAT TYPE
# -----------------------------------------------------------
def get_seat_type():
    seat = input("Enter seating type (chair / bench): ").strip().lower()
    if seat == "chair":
        return 0.5, 0.5
    else:
        return 1.2, 0.5  # bench size

# -----------------------------------------------------------
# CREATE RECTANGULAR ROOM FROM USER SIZE
# -----------------------------------------------------------
def create_room_polygon():
    L = float(input("Enter room length (meters): "))
    W = float(input("Enter room width (meters): "))
    return [(0, 0), (L, 0), (L, W), (0, W)], L, W

# -----------------------------------------------------------
# GET DOORS WITH SIDE + POSITION
# -----------------------------------------------------------
def get_doors():
    doors = []
    n = int(input("Enter number of doors: "))
    for i in range(n):
        print(f"\nDoor {i+1}")
        side = input("Side of door (left/right/top/bottom): ").strip().lower()
        pos = float(input("Distance from corner along that side (meters): "))
        doors.append((side, pos))
    return doors

# -----------------------------------------------------------
# GENERATE SEATING PLAN
# -----------------------------------------------------------
def generate_seating(
    room_points, bench_w, bench_h, spacing, aisle_space, doors,
    board_side, total_students, output_file
):
    room_poly = Polygon(room_points)
    minx, miny, maxx, maxy = room_poly.bounds

    # -----------------------------------------------------------
    # BOARD POSITION
    # -----------------------------------------------------------
    board_length = maxx - minx if board_side in ["top", "bottom"] else maxy - miny
    board_height = 0.4

    if board_side == "top":
        board_x, board_y = minx, maxy - board_height
    elif board_side == "bottom":
        board_x, board_y = minx, miny
    elif board_side == "left":
        board_x, board_y = minx, miny
    else:  # right
        board_x, board_y = maxx - board_height, miny

    # -----------------------------------------------------------
    # TEACHER TABLE POSITION (fixed, avoid door)
    # -----------------------------------------------------------
    table_w, table_h = 1.2, 0.6
    table_distance = 0.5

    # Initial placement near board
    if board_side == "top":
        table_x = (maxx - minx)/2 - table_w/2
        table_y = board_y - table_distance - table_h
    elif board_side == "bottom":
        table_x = (maxx - minx)/2 - table_w/2
        table_y = board_y + board_height + table_distance
    elif board_side == "left":
        table_x = board_x + board_height + table_distance
        table_y = (maxy - miny)/2 - table_h/2
    else:  # right
        table_x = board_x - table_w - table_distance
        table_y = (maxy - miny)/2 - table_h/2

    table_rect = box(table_x, table_y, table_x + table_w, table_y + table_h)

    # -----------------------------------------------------------
    # DOOR RESTRICTION AREAS
    # -----------------------------------------------------------
    door_clear_w, door_clear_h = 1.2, 2.0
    door_rects = []
    for side, pos in doors:
        if side == "left":
            rect = box(minx, pos, minx + door_clear_w, pos + door_clear_h)
        elif side == "right":
            rect = box(maxx - door_clear_w, pos, maxx, pos + door_clear_h)
        elif side == "top":
            rect = box(pos, maxy - door_clear_w, pos + door_clear_h, maxy)
        else:  # bottom
            rect = box(pos, miny, pos + door_clear_h, miny + door_clear_w)
        door_rects.append(rect)

    # -----------------------------------------------------------
    # ADJUST TABLE IF OVERLAPPING DOOR
    # -----------------------------------------------------------
    def adjust_table(table_rect):
        shift = 0.1  # incremental shift
        max_attempts = 50
        attempts = 0
        while any(table_rect.intersects(dr) for dr in door_rects) and attempts < max_attempts:
            if board_side in ["top", "bottom"]:
                # move table left if possible, else right
                if table_rect.bounds[0] - shift >= minx + aisle_space:
                    table_rect = box(table_rect.bounds[0]-shift, table_rect.bounds[1],
                                     table_rect.bounds[2]-shift, table_rect.bounds[3])
                elif table_rect.bounds[2] + shift <= maxx - aisle_space:
                    table_rect = box(table_rect.bounds[0]+shift, table_rect.bounds[1],
                                     table_rect.bounds[2]+shift, table_rect.bounds[3])
            else:
                # move table up if possible, else down
                if table_rect.bounds[1] - shift >= miny + aisle_space:
                    table_rect = box(table_rect.bounds[0], table_rect.bounds[1]-shift,
                                     table_rect.bounds[2], table_rect.bounds[3]-shift)
                elif table_rect.bounds[3] + shift <= maxy - aisle_space:
                    table_rect = box(table_rect.bounds[0], table_rect.bounds[1]+shift,
                                     table_rect.bounds[2], table_rect.bounds[3]+shift)
            attempts += 1
        return table_rect

    table_rect = adjust_table(table_rect)

    # -----------------------------------------------------------
    # GENERATE SEATING GRID
    # -----------------------------------------------------------
    col_pitch = bench_w + spacing
    row_pitch = bench_h + spacing
    possible_seats = []

    if board_side in ["top", "bottom"]:
        y_start = maxy - aisle_space - bench_h if board_side == "top" else miny + aisle_space
        y_end = miny + aisle_space if board_side == "top" else maxy - aisle_space - bench_h
        y = y_start
        while (y >= y_end if board_side == "top" else y <= y_end):
            x = minx + aisle_space
            while x + bench_w <= maxx - aisle_space:
                seat_rect = box(x, y, x + bench_w, y + bench_h)
                blocked = seat_rect.intersects(table_rect) or any(seat_rect.intersects(dr) for dr in door_rects)
                if not blocked and room_poly.contains(seat_rect):
                    possible_seats.append((x, y))
                x += col_pitch
            y = y - row_pitch if board_side == "top" else y + row_pitch
    else:  # left/right board
        x_start = minx + aisle_space if board_side == "left" else maxx - aisle_space - bench_w
        x_end = maxx - aisle_space - bench_w if board_side == "left" else minx + aisle_space
        x = x_start
        while (x <= x_end if board_side == "left" else x >= x_end):
            y = miny + aisle_space
            while y + bench_h <= maxy - aisle_space:
                seat_rect = box(x, y, x + bench_w, y + bench_h)
                blocked = seat_rect.intersects(table_rect) or any(seat_rect.intersects(dr) for dr in door_rects)
                if not blocked and room_poly.contains(seat_rect):
                    possible_seats.append((x, y))
                y += row_pitch
            x = x + col_pitch if board_side == "left" else x - col_pitch

    # -----------------------------------------------------------
    # ASSIGN ROLL NUMBERS
    # -----------------------------------------------------------
    seating_map = {}
    roll = 1
    row_groups = {}
    for x, y in possible_seats:
        row_groups.setdefault(y, []).append((x, y))
    sorted_rows = sorted(row_groups.keys(), reverse=(board_side in ["top", "right"]))
    for row_y in sorted_rows:
        seats = sorted(row_groups[row_y], key=lambda s: s[0])
        for seat in seats:
            seating_map[seat] = roll if roll <= total_students else None
            roll += 1

    # -----------------------------------------------------------
    # DRAW FINAL LAYOUT
    # -----------------------------------------------------------
    fig, ax = plt.subplots(figsize=(12, 9))
    ax.add_patch(Rectangle((minx, miny), maxx - minx, maxy - miny, fill=False, linewidth=2))  # Room

    # Board
    if board_side in ["top", "bottom"]:
        ax.add_patch(Rectangle((board_x, board_y), board_length, board_height, color="black", alpha=0.5))
        ax.text(board_x + board_length/2, board_y + board_height/2, "BOARD", color="white", ha="center", va="center", fontsize=10)
    else:
        ax.add_patch(Rectangle((board_x, board_y), board_height, board_length, color="black", alpha=0.5))
        ax.text(board_x + board_height/2, board_y + board_length/2, "BOARD", color="white", ha="center", va="center", fontsize=10)

    # Table
    ax.add_patch(Rectangle((table_rect.bounds[0], table_rect.bounds[1]), table_w, table_h, color="brown", alpha=0.6))
    ax.text(table_rect.bounds[0] + table_w/2, table_rect.bounds[1] + table_h/2, "Table", ha="center", va="center", fontsize=9, color="white")

    # Doors
    for dr in door_rects:
        dx, dy, dx2, dy2 = dr.bounds
        ax.add_patch(Rectangle((dx, dy), dx2 - dx, dy2 - dy, color="green", alpha=0.4))
        ax.text(dx + (dx2 - dx)/2, dy + 0.3, "Door", ha="center", fontsize=8)

    # Seats
    for (x, y), roll in seating_map.items():
        ax.add_patch(Rectangle((x, y), bench_w, bench_h, fill=False, color="blue"))
        if roll:
            ax.text(x + bench_w/2, y + bench_h/2, str(roll), ha="center", va="center")
        if board_side == "top":
            ax.add_patch(FancyArrow(x + bench_w/2, y + bench_h, 0, 0.3, width=0.05))
        elif board_side == "bottom":
            ax.add_patch(FancyArrow(x + bench_w/2, y, 0, -0.3, width=0.05))
        elif board_side == "left":
            ax.add_patch(FancyArrow(x, y + bench_h/2, 0.3, 0, width=0.05))
        else:  # right
            ax.add_patch(FancyArrow(x + bench_w, y + bench_h/2, -0.3, 0, width=0.05))

    plt.title("Exam Seating Plan (Roll starts near board)")
    plt.axis("equal")
    plt.axis("off")
    plt.savefig(output_file, dpi=300)
    plt.close()
    print("Seating plan saved as:", output_file)
    print("Total seating positions:", len(possible_seats))

# -----------------------------------------------------------
# MAIN
# -----------------------------------------------------------
if __name__ == "__main__":
    room_polygon, L, W = create_room_polygon()
    bench_w, bench_h = get_seat_type()
    students = int(input("Enter number of students: "))
    spacing = 0.3
    aisle = 1.0
    board_side = input("Board side (left/right/top/bottom): ").strip().lower()
    doors = get_doors()
    generate_seating(
        room_points=room_polygon,
        bench_w=bench_w,
        bench_h=bench_h,
        spacing=spacing,
        aisle_space=aisle,
        doors=doors,
        board_side=board_side,
        total_students=students,
        output_file="classroom_seating_fixed_table.png"
    )


Enter room length (meters): 12
Enter room width (meters): 11
Enter seating type (chair / bench): chair
Enter number of students: 8
Board side (left/right/top/bottom): top
Enter number of doors: 0
Seating plan saved as: classroom_seating_fixed_table.png
Total seating positions: 129


Here, the plan is arranged based on the number of students.Here, the seats are arranged based on the number of students, and the distances between them are adjusted automatically.



In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from shapely.geometry import Polygon, box
import math

# ----------------------------
# CONFIGURABLE MIN/MAX VALUES
# ----------------------------
PREFERRED_MIN_SPACING = 0.20
FALLBACK_MIN_SPACING = 0.02
BOARD_THICKNESS = 0.4
DOOR_W = 1.2
DOOR_H = 2.0
TABLE_DEFAULT_W = 1.2
TABLE_DEFAULT_H = 0.6
TABLE_MIN_W = 0.8
TABLE_MIN_H = 0.4

# ----------------------------
# USER INPUT FUNCTIONS
# ----------------------------
def get_seat_type():
    seat = input("Enter seating type (chair / bench): ").strip().lower()
    return (0.5, 0.5) if seat == "chair" else (1.2, 0.5)

def create_room_polygon():
    L = float(input("Enter room length (meters): "))
    W = float(input("Enter room width (meters): "))
    return [(0, 0), (L, 0), (L, W), (0, W)], L, W

def get_doors():
    doors = []
    n = int(input("Enter number of doors: "))
    for i in range(n):
        print(f"\nDoor {i+1}")
        side = input("Side of door (left/right/top/bottom): ").strip().lower()
        pos = float(input("Distance from corner along that side (meters): "))
        doors.append((side, pos))
    return doors

def build_door_rects(doors, minx, miny, maxx, maxy):
    door_rects = []
    for side, pos in doors:
        if side == "left":
            dr = box(minx, pos, minx + DOOR_W, pos + DOOR_H)
        elif side == "right":
            dr = box(maxx - DOOR_W, pos, maxx, pos + DOOR_H)
        elif side == "top":
            dr = box(pos, maxy - DOOR_W, pos + DOOR_H, maxy)
        else:  # bottom
            dr = box(pos, miny, pos + DOOR_H, miny + DOOR_W)
        door_rects.append(dr)
    return door_rects

# ----------------------------
# TABLE FUNCTIONS
# ----------------------------
def initial_table_rect(board_side, minx, miny, maxx, maxy):
    table_w = TABLE_DEFAULT_W
    table_h = TABLE_DEFAULT_H
    table_gap = 0.5

    if board_side == "top":
        table_x = (minx + maxx) / 2 - table_w / 2
        table_y = maxy - BOARD_THICKNESS - table_h - table_gap
    elif board_side == "bottom":
        table_x = (minx + maxx) / 2 - table_w / 2
        table_y = miny + BOARD_THICKNESS + table_gap
    elif board_side == "left":
        table_x = minx + BOARD_THICKNESS + table_gap
        table_y = (miny + maxy) / 2 - table_h / 2
    else:
        table_x = maxx - BOARD_THICKNESS - table_gap - table_w
        table_y = (miny + maxy) / 2 - table_h / 2

    return box(table_x, table_y, table_x + table_w, table_y + table_h), table_w, table_h

def move_table_away_if_blocking(table_rect, table_w, table_h, door_rects, board_side, minx, miny, maxx, maxy):
    shift = 0.1
    max_attempts = 50
    attempt = 0
    while any(table_rect.intersects(dr) for dr in door_rects) and attempt < max_attempts:
        attempt += 1
        if board_side in ("top", "bottom"):
            if table_rect.bounds[0] > minx:
                table_rect = box(table_rect.bounds[0]-shift, table_rect.bounds[1],
                                 table_rect.bounds[2]-shift, table_rect.bounds[3])
            else:
                table_rect = box(table_rect.bounds[0]+shift, table_rect.bounds[1],
                                 table_rect.bounds[2]+shift, table_rect.bounds[3])
        else:
            if table_rect.bounds[1] > miny:
                table_rect = box(table_rect.bounds[0], table_rect.bounds[1]-shift,
                                 table_rect.bounds[2], table_rect.bounds[3]-shift)
            else:
                table_rect = box(table_rect.bounds[0], table_rect.bounds[1]+shift,
                                 table_rect.bounds[2], table_rect.bounds[3]+shift)
    return table_rect, table_w, table_h

# ----------------------------
# LAYOUT GENERATION
# ----------------------------
def find_layout_with_compression(room_poly, minx, miny, maxx, maxy,
                                 bench_w, bench_h, door_rects, table_rect,
                                 board_side, total_students):
    usable_width = maxx - minx
    usable_height = maxy - miny

    start_cols = max(1, math.ceil(math.sqrt(total_students)))
    for cols in range(start_cols, total_students + 1):
        rows = math.ceil(total_students / cols)
        col_spacing_ideal = (usable_width - cols * bench_w) / (cols + 1)
        row_spacing_ideal = (usable_height - rows * bench_h) / (rows + 1)

        spacing_candidates = [
            (col_spacing_ideal, row_spacing_ideal),
            (max(col_spacing_ideal, PREFERRED_MIN_SPACING), max(row_spacing_ideal, PREFERRED_MIN_SPACING)),
            (max(col_spacing_ideal, FALLBACK_MIN_SPACING), max(row_spacing_ideal, FALLBACK_MIN_SPACING)),
            (max(col_spacing_ideal, 0.0), max(row_spacing_ideal, 0.0))
        ]

        tried = set()
        for col_spacing, row_spacing in spacing_candidates:
            key = (round(col_spacing, 4), round(row_spacing, 4))
            if key in tried:
                continue
            tried.add(key)

            req_w = cols * bench_w + (cols + 1) * col_spacing
            req_h = rows * bench_h + (rows + 1) * row_spacing
            if req_w - 1e-6 > usable_width or req_h - 1e-6 > usable_height:
                continue

            candidates = []
            for r in range(rows):
                for c in range(cols):
                    if board_side in ("top", "bottom"):
                        x = minx + col_spacing + c * (bench_w + col_spacing)
                        if board_side == "top":
                            y = maxy - BOARD_THICKNESS - row_spacing - bench_h - r * (bench_h + row_spacing)
                        else:
                            y = miny + BOARD_THICKNESS + row_spacing + r * (bench_h + row_spacing)
                    else:
                        y = miny + row_spacing + c * (bench_h + row_spacing)
                        if board_side == "left":
                            x = minx + BOARD_THICKNESS + col_spacing + r * (bench_w + col_spacing)
                        else:
                            x = maxx - BOARD_THICKNESS - col_spacing - bench_w - r * (bench_w + col_spacing)
                    seat_rect = box(x, y, x + bench_w, y + bench_h)
                    candidates.append((x, y, seat_rect))

            valid = []
            for x, y, rect in candidates:
                if not room_poly.contains(rect):
                    continue
                if rect.intersects(table_rect):
                    continue
                if any(rect.intersects(dr) for dr in door_rects):
                    continue
                valid.append((x, y))

            if len(valid) >= total_students:
                chosen = [(vx, vy, idx + 1) for idx, (vx, vy) in enumerate(valid[:total_students])]
                return {
                    "seating_positions": chosen,
                    "cols": cols,
                    "rows": rows,
                    "col_spacing": col_spacing,
                    "row_spacing": row_spacing
                }
    return None

# ----------------------------
# MAIN GENERATE FUNCTION
# ----------------------------
def generate_seating(room_points, bench_w, bench_h, doors, board_side, total_students,
                     output_file):
    room_poly = Polygon(room_points)
    minx, miny, maxx, maxy = room_poly.bounds
    door_rects = build_door_rects(doors, minx, miny, maxx, maxy)
    table_rect_init, table_w, table_h = initial_table_rect(board_side, minx, miny, maxx, maxy)
    table_rect, table_w, table_h = move_table_away_if_blocking(
        table_rect_init, table_w, table_h, door_rects, board_side, minx, miny, maxx, maxy)

    layout = find_layout_with_compression(room_poly, minx, miny, maxx, maxy,
                                          bench_w, bench_h, door_rects, table_rect,
                                          board_side, total_students)

    seating_positions = layout["seating_positions"] if layout else []

    # ----------------------------
    # PLOT SEATING PLAN ONLY
    # ----------------------------
    fig, ax = plt.subplots(figsize=(12, 9))
    ax.add_patch(Rectangle((minx, miny), maxx - minx, maxy - miny, fill=False, linewidth=2))

    # Board
    if board_side == "top":
        board_x, board_y = minx, maxy - BOARD_THICKNESS
        board_w, board_h = maxx - minx, BOARD_THICKNESS
    elif board_side == "bottom":
        board_x, board_y = minx, miny
        board_w, board_h = maxx - minx, BOARD_THICKNESS
    elif board_side == "left":
        board_x, board_y = minx, miny
        board_w, board_h = BOARD_THICKNESS, maxy - miny
    else:
        board_x, board_y = maxx - BOARD_THICKNESS, miny
        board_w, board_h = BOARD_THICKNESS, maxy - miny
    ax.add_patch(Rectangle((board_x, board_y), board_w, board_h, color="black", alpha=0.8))
    ax.text(board_x + board_w/2, board_y + board_h/2, "BOARD", color="white", ha="center", va="center")

    # Table
    tx1, ty1, tx2, ty2 = table_rect.bounds
    ax.add_patch(Rectangle((tx1, ty1), tx2-tx1, ty2-ty1, color="saddlebrown", alpha=0.9))
    ax.text(tx1 + (tx2-tx1)/2, ty1 + (ty2-ty1)/2, "Table", color="white", ha="center", va="center", fontsize=8)

    # Doors
    for dr in door_rects:
        dx1, dy1, dx2, dy2 = dr.bounds
        ax.add_patch(Rectangle((dx1, dy1), dx2-dx1, dy2-dy1, color="green", alpha=0.5))
        ax.text(dx1 + (dx2-dx1)/2, dy1 + 0.12, "Door", ha="center", fontsize=7)

    # Seats
    for x, y, roll_no in seating_positions:
        ax.add_patch(Rectangle((x, y), bench_w, bench_h, fill=False, edgecolor="blue"))
        ax.text(x + bench_w/2, y + bench_h/2, str(roll_no), ha="center", va="center", fontsize=8)

    plt.title("Seating Plan")
    plt.axis("equal")
    plt.axis("off")
    plt.savefig(output_file, dpi=300)
    plt.close()
    print("✔ Seating Plan Saved:", output_file)
    print("✔ Total students requested:", total_students)
    print("✔ Total students placed:", len(seating_positions))

# ----------------------------
# MAIN
# ----------------------------
if __name__ == "__main__":
    room_polygon, L, W = create_room_polygon()
    bench_w, bench_h = get_seat_type()
    students = int(input("Enter number of students: "))
    board_side = input("Board side (top/bottom/left/right): ").strip().lower()
    doors = get_doors()

    generate_seating(
        room_points=room_polygon,
        bench_w=bench_w,
        bench_h=bench_h,
        doors=doors,
        board_side=board_side,
        total_students=students,
        output_file="classroom_seating.png"
    )


Enter room length (meters): 10
Enter room width (meters): 10
Enter seating type (chair / bench): chair
Enter number of students: 4
Board side (top/bottom/left/right): top
Enter number of doors: 1

Door 1
Side of door (left/right/top/bottom): top
Distance from corner along that side (meters): 5
✔ Seating Plan Saved: classroom_seating.png
✔ Total students requested: 4
✔ Total students placed: 4
